In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import Row
from pyspark.sql.functions import max as spark_max
from delta.tables import DeltaTable
import os

# -------------------
# Spark session
# -------------------
spark = (
    SparkSession.builder
        .appName("Delta S3 Hive Auto Insert")
        .master("local[*]")
        .config("spark.jars.packages",
                "io.delta:delta-spark_2.13:4.0.0,org.apache.hadoop:hadoop-aws:3.4.1")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        # MinIO / S3
        .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
        .config("spark.hadoop.fs.s3a.access.key", "admin")
        .config("spark.hadoop.fs.s3a.secret.key", "password")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
        .config("spark.sql.warehouse.dir", "s3a://warehouse/")
        # Hive Metastore
        .config("spark.hadoop.hive.metastore.uris", "thrift://localhost:9083")
        .config("spark.sql.catalogImplementation", "hive")
        .getOrCreate()
)

print("Spark session started")

# -------------------
# Create Hive database if missing
# -------------------
spark.sql("CREATE DATABASE IF NOT EXISTS silver")

# -------------------
# Delta table path
# -------------------
table_name = "silver.silver_ecg"
table_path = "s3a://warehouse/silver.db/silver_ecg"

Spark session started


In [2]:
spark.sql("SHOW CATALOGS").show(truncate=False)

+-------------+
|catalog      |
+-------------+
|spark_catalog|
+-------------+



In [3]:
# Check current catalog
current_catalog = spark.catalog.currentCatalog()
print("Current catalog:", current_catalog)

Current catalog: spark_catalog


In [9]:
# spark.sql("DROP TABLE IF EXISTS silver.silver_ecg")

DataFrame[]

In [4]:
from pyspark.sql import Row
from pyspark.sql.functions import current_timestamp, col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType

# ---------- Define schema ----------
ecg_schema = StructType([
    StructField("device_id", StringType(), True),
    StructField("record_id", StringType(), True),
    StructField("start_time", StringType(), True),
    StructField("ts", DoubleType(), True),
    StructField("ecg_i", DoubleType(), True),
    StructField("ecg_i_filtered", DoubleType(), True),
    StructField("file_modified_ts", StringType(), True),
    StructField("ingest_ts", TimestampType(), True)
])

# ---------- Create sample input ----------
sample_row = [("D_sample", "rec_sample", "2025-12-28T08:29:17", 10.0, 0.02, 0.095, None)]
df_sample = spark.createDataFrame(sample_row, schema=ecg_schema)

# Add current ingest timestamp
df_sample = df_sample.withColumn("ingest_ts", current_timestamp())

# ---------- Set database ----------
spark.catalog.setCurrentDatabase("silver")

# ---------- Write to Delta table ----------
table_name = "silver.silver_ecg"
df_sample.write.format("delta").mode("append").saveAsTable(table_name)

# ---------- Display ----------
spark.sql(f"SELECT * FROM {table_name}").show(5, truncate=False)
print("Total rows:", spark.sql(f"SELECT COUNT(*) FROM {table_name}").collect()[0][0])


25/12/28 21:41:11 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to silver.db/silver_ecg/part-00000-82aa193e-58ba-431a-ad99-109355ee9051-c000.snappy.parquet. This is Unsupported
25/12/28 21:41:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/12/28 21:41:17 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`silver`.`silver_ecg` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
25/12/28 21:41:17 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


+---------+----------+-------------------+----+-----+--------------+--------------------------+
|device_id|record_id |start_time         |ts  |ecg_i|ecg_i_filtered|ingest_ts                 |
+---------+----------+-------------------+----+-----+--------------+--------------------------+
|D_sample |rec_sample|2025-12-28T08:29:17|10.0|0.02 |0.095         |2025-12-28 21:41:07.829835|
+---------+----------+-------------------+----+-----+--------------+--------------------------+

Total rows: 1


In [5]:
# Show first 20 rows
spark.sql("SELECT * FROM silver.silver_ecg").show(truncate=False)

# Count rows
spark.sql("SELECT COUNT(*) FROM silver.silver_ecg").show()

+---------+----------+-------------------+----+-----+--------------+--------------------------+
|device_id|record_id |start_time         |ts  |ecg_i|ecg_i_filtered|ingest_ts                 |
+---------+----------+-------------------+----+-----+--------------+--------------------------+
|D_sample |rec_sample|2025-12-28T08:29:17|10.0|0.02 |0.095         |2025-12-28 21:41:07.829835|
+---------+----------+-------------------+----+-----+--------------+--------------------------+

+--------+
|count(1)|
+--------+
|       1|
+--------+



In [6]:
# Check current catalog
current_catalog = spark.catalog.currentCatalog()
print("Current catalog:", current_catalog)

Current catalog: spark_catalog


In [7]:
# Show detailed info including location
spark.sql("DESCRIBE DETAIL silver.silver_ecg").show(truncate=False)

+------+------------------------------------+-------------------------------+-----------+------------------------------------+-----------------------+-------------------+----------------+-----------------+--------+-----------+----------+----------------+----------------+------------------------+
|format|id                                  |name                           |description|location                            |createdAt              |lastModified       |partitionColumns|clusteringColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|tableFeatures           |
+------+------------------------------------+-------------------------------+-----------+------------------------------------+-----------------------+-------------------+----------------+-----------------+--------+-----------+----------+----------------+----------------+------------------------+
|delta |a5f04c28-015e-468a-bbb6-c6d558e5398f|spark_catalog.silver.silver_ecg|NULL       |s3a://warehouse/silv